In [0]:
from google.cloud import language
from google.oauth2 import service_account
from google.cloud.language import enums
from google.cloud.language import types

In [0]:
#Creating an instance with the API/webservice credentials
client = language.LanguageServiceClient.from_service_account_json('/content/My First Project-3f8671cb92c1.json')

In [22]:
#Importing the data
import pandas as pd
data = pd.read_excel('/content/US_IHG V4.xlsx', sheet_name = 'data')
data

,HB_SOCIAL_INSIGHT_REVIEW_ID,STAR_HTL_NM,INDVL_REVIEW_SCOR_NBR,REVIEW_TXT
0,BKG-20190696110,Crowne Plaza Times Square Manhattan,4.8,Comfortable room. great location. good facilities
1,BKG-201909713799,Crowne Plaza Times Square Manhattan,5,Very good location great staff and great exper...
2,GGL-20190696100,Crowne Plaza Times Square Manhattan,4.4,Refrigerator.
3,BKG-201909713895,Crowne Plaza Times Square Manhattan,2.5,Just the location. They didnt want to honor th...
4,GGL-20190696129,Crowne Plaza Times Square Manhattan,4.8,Great location. Better cancellation policy.
...,...,...,...,...
276,BKG-20190686420,Holiday Inn Express Chester,5,Everything thing about bit was fantastic .No i...
277,BKG-201909761650,Holiday Inn Express Chester,4,Plenty of hot water good powerful shower and l...
278,BKG-20190686863,Holiday Inn Express Chester,4.6,Location was great overlooking the racecourse ...
279,BKG-201909761663,Holiday Inn Express Chester,4.4,The location and access to Chester City Centre...


In [26]:
d1 = data.head(20)[['HB_SOCIAL_INSIGHT_REVIEW_ID', 'REVIEW_TXT']]
d1.head()

,HB_SOCIAL_INSIGHT_REVIEW_ID,REVIEW_TXT
0,BKG-20190696110,Comfortable room. great location. good facilities
1,BKG-201909713799,Very good location great staff and great exper...
2,GGL-20190696100,Refrigerator.
3,BKG-201909713895,Just the location. They didnt want to honor th...
4,GGL-20190696129,Great location. Better cancellation policy.


In [0]:
def get_entities(text):
  #Taking input 
  document = language.types.Document(
      content = text,
      type=language.enums.Document.Type.PLAIN_TEXT,
  )
  #Using the analyse_entites functions
  response = client.analyze_entities(
      document=document,
     encoding_type='UTF32',
  )
  #creating a list that stores entities found at each level
  lst_ent = []  
  for i in response.entities:
    lst_ent.append((i.name, i.type, i.salience))
  return lst_ent

In [0]:
def get_entities_senti(text):
    #Taking input 
  document = language.types.Document(
      content = text,
      type=language.enums.Document.Type.PLAIN_TEXT,
  )
    #Using the analyse_entity_sentiment function to get entity and sentiment
  response = client.analyze_entity_sentiment(
      document=document,
     encoding_type='UTF32',
  )
  #creating a list that stores entities & their scores found at each level
  score_lst = [] 
  for i in response.entities:
    score_lst.append((i.name,i.sentiment.score))
  return score_lst

In [66]:
d2 = data[['HB_SOCIAL_INSIGHT_REVIEW_ID', 'REVIEW_TXT']]

#Using the entity_sentiment function to get entities and their respective scores against each Review
d2['entity_sentiment'] = d2.REVIEW_TXT.map(str).apply(lambda x: get_entities_senti(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,HB_SOCIAL_INSIGHT_REVIEW_ID,REVIEW_TXT,entity_sentiment
0,BKG-20190696110,Comfortable room. great location. good facilities,"[(room, 0.8999999761581421), (location, 0.8999..."
1,BKG-201909713799,Very good location great staff and great exper...,"[(location, 0.8999999761581421), (staff, 0.899..."
2,GGL-20190696100,Refrigerator.,"[(Refrigerator, 0.0)]"
3,BKG-201909713895,Just the location. They didnt want to honor th...,"[(time, 0.0), (location, 0.0), (military rate,..."
4,GGL-20190696129,Great location. Better cancellation policy.,"[(location, 0.8999999761581421), (cancellation..."
...,...,...,...
276,BKG-20190686420,Everything thing about bit was fantastic .No i...,"[(thing, 0.10000000149011612), (bit, 0.5)]"
277,BKG-201909761650,Plenty of hot water good powerful shower and l...,"[(water, 0.8999999761581421), (shower, 0.89999..."
278,BKG-20190686863,Location was great overlooking the racecourse ...,"[(racecourse, 0.6000000238418579), (Location, ..."
279,BKG-201909761663,The location and access to Chester City Centre...,"[(location, 0.0), (access, 0.20000000298023224..."


In [0]:
#Breaking up multiple entities against a single review into multiple rows
d3 = d2.set_index('HB_SOCIAL_INSIGHT_REVIEW_ID').entity_sentiment.apply(pd.Series).stack().reset_index(name = 'entity')

In [0]:
#Breaking up the stacked entities into two columns - 'entity' and 'score'
for index,i in enumerate(d3.entity):
  d3.loc[index,'entity_word'] = i[0]
  d3.loc[index,'entity_word_sentiment'] = i[1]

In [90]:
#Bring in the sentence against each ID
d4 = d3.merge(d2[['HB_SOCIAL_INSIGHT_REVIEW_ID','REVIEW_TXT']])

#Creating an ID for multiple entites
d4['ID_SID'] = d4['HB_SOCIAL_INSIGHT_REVIEW_ID'] + "_" + (d4['level_1']+1).map(str)

d4 = d4.drop(columns = 'level_1')
d4 = d4.reindex(columns = ['ID_SID', 'REVIEW_TXT', 'entity', 'entity_word',
       'entity_word_sentiment' ])


,HB_SOCIAL_INSIGHT_REVIEW_ID,level_1,entity,entity_word,entity_word_sentiment,REVIEW_TXT
0,BKG-20190696110,0,"(room, 0.8999999761581421)",room,0.9,Comfortable room. great location. good facilities
1,BKG-20190696110,1,"(location, 0.8999999761581421)",location,0.9,Comfortable room. great location. good facilities
2,BKG-20190696110,2,"(facilities, 0.8999999761581421)",facilities,0.9,Comfortable room. great location. good facilities
3,BKG-201909713799,0,"(location, 0.8999999761581421)",location,0.9,Very good location great staff and great exper...
4,BKG-201909713799,1,"(staff, 0.8999999761581421)",staff,0.9,Very good location great staff and great exper...


In [0]:
w = pd.ExcelWriter('OP_v1.xlsx')
d1.to_excel(w, sheet_name = 'entities_RevLevel')
d2.to_excel(w, sheet_name = 'ID_Rev_Entity_send')
d3.to_excel(w, sheet_name = 'entities_RevLevel')
d4.to_excel(w, sheet_name = 'entities_RevLevel_v1')
w.save()

##Sentence Level Entities

In [106]:
data['split_rev'] = data.REVIEW_TXT.apply(lambda x: x.split("."))
data.head()

,HB_SOCIAL_INSIGHT_REVIEW_ID,STAR_HTL_NM,INDVL_REVIEW_SCOR_NBR,REVIEW_TXT,split_rev
0,BKG-20190696110,Crowne Plaza Times Square Manhattan,4.8,Comfortable room. great location. good facilities,"[Comfortable room, great location, good faci..."
1,BKG-201909713799,Crowne Plaza Times Square Manhattan,5,Very good location great staff and great exper...,[Very good location great staff and great expe...
2,GGL-20190696100,Crowne Plaza Times Square Manhattan,4.4,Refrigerator.,"[Refrigerator, ]"
3,BKG-201909713895,Crowne Plaza Times Square Manhattan,2.5,Just the location. They didnt want to honor th...,"[Just the location, They didnt want to honor ..."
4,GGL-20190696129,Crowne Plaza Times Square Manhattan,4.8,Great location. Better cancellation policy.,"[Great location, Better cancellation policy, ]"


In [0]:
data_s = data.set_index('HB_SOCIAL_INSIGHT_REVIEW_ID').split_rev.apply(pd.Series).stack().reset_index(name = 'Sentence')

In [111]:
data_s['ID_SID'] = data_s['HB_SOCIAL_INSIGHT_REVIEW_ID'] + "_" + (data_s['level_1'] + 1).map(str)


,HB_SOCIAL_INSIGHT_REVIEW_ID,Sentence,ID_SID
0,BKG-20190696110,Comfortable room,BKG-20190696110_1
1,BKG-20190696110,great location,BKG-20190696110_2
2,BKG-20190696110,good facilities,BKG-20190696110_3
3,BKG-201909713799,Very good location great staff and great exper...,BKG-201909713799_1
4,BKG-201909713799,Honestly nothings,BKG-201909713799_2
...,...,...,...
1170,GGL-20190685608,The breakfast was fine if you like a cooked b...,GGL-20190685608_3
1171,GGL-20190685608,Also the bananas were very over ripe,GGL-20190685608_4
1172,GGL-20190685608,And most importantly no de caff tea bags,GGL-20190685608_5
1173,GGL-20190685608,However overall a great stay for a decent price,GGL-20190685608_6


In [0]:
data_s = data_s.drop(columns = 'level_1')

In [116]:
data_s.merge(d2[['HB_SOCIAL_INSIGHT_REVIEW_ID','REVIEW_TXT']]).reindex(columns = ['HB_SOCIAL_INSIGHT_REVIEW_ID',
                                                                                'REVIEW_TXT','ID_SID','Sentence'])



,HB_SOCIAL_INSIGHT_REVIEW_ID,REVIEW_TXT,ID_SID,Sentence
0,BKG-20190696110,Comfortable room. great location. good facilities,BKG-20190696110_1,Comfortable room
1,BKG-20190696110,Comfortable room. great location. good facilities,BKG-20190696110_2,great location
2,BKG-20190696110,Comfortable room. great location. good facilities,BKG-20190696110_3,good facilities
3,BKG-201909713799,Very good location great staff and great exper...,BKG-201909713799_1,Very good location great staff and great exper...
4,BKG-201909713799,Very good location great staff and great exper...,BKG-201909713799_2,Honestly nothings
...,...,...,...,...
1170,GGL-20190685608,Staff very friendly and helpful. Nice size roo...,GGL-20190685608_3,The breakfast was fine if you like a cooked b...
1171,GGL-20190685608,Staff very friendly and helpful. Nice size roo...,GGL-20190685608_4,Also the bananas were very over ripe
1172,GGL-20190685608,Staff very friendly and helpful. Nice size roo...,GGL-20190685608_5,And most importantly no de caff tea bags
1173,GGL-20190685608,Staff very friendly and helpful. Nice size roo...,GGL-20190685608_6,However overall a great stay for a decent price


In [0]:
data_s1 = data_s.merge(d2[['HB_SOCIAL_INSIGHT_REVIEW_ID','REVIEW_TXT']]).reindex(columns = ['HB_SOCIAL_INSIGHT_REVIEW_ID',
                                                                                'REVIEW_TXT','ID_SID','Sentence'])


In [0]:
# data_s1['entity_score_400'] = data_s1.Sentence[:400].apply(lambda x: get_entities_senti(x))
# data_s1


data_s1_1 = data_s1.iloc[:300, :]
data_s1_2 = data_s1.iloc[300:600, :]
data_s1_3 = data_s1.iloc[600:900, :]
data_s1_4 = data_s1.iloc[900:1175, :]


In [143]:
# data_s1_1['entity_score'] = data_s1_1.Sentence.map(str).apply(lambda x: get_entities_senti(x))
# data_s1_1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,HB_SOCIAL_INSIGHT_REVIEW_ID,REVIEW_TXT,ID_SID,Sentence,entity_score,entity_word
0,BKG-20190696110,Comfortable room. great location. good facilities,BKG-20190696110_1,Comfortable room,"[(room, 0.8999999761581421)]",NaN
1,BKG-20190696110,Comfortable room. great location. good facilities,BKG-20190696110_2,great location,"[(location, 0.8999999761581421)]",NaN
2,BKG-20190696110,Comfortable room. great location. good facilities,BKG-20190696110_3,good facilities,"[(facilities, 0.8999999761581421)]",NaN
3,BKG-201909713799,Very good location great staff and great exper...,BKG-201909713799_1,Very good location great staff and great exper...,"[(location, 0.8999999761581421), (staff, 0.899...",NaN
4,BKG-201909713799,Very good location great staff and great exper...,BKG-201909713799_2,Honestly nothings,"[(nothings, -0.8999999761581421)]",NaN
...,...,...,...,...,...,...
295,BKG-201909713816,The location was perfect direct on Broadway an...,BKG-201909713816_3,,[],NaN
296,GGL-20190696111,Clean comfortable place in good location nothi...,GGL-20190696111_1,Clean comfortable place in good location nothi...,"[(place, 0.8999999761581421), (location nothin...",NaN
297,GGL-20190696111,Clean comfortable place in good location nothi...,GGL-20190696111_2,,[],NaN
298,BKG-201909713873,Everything the location was great the staff we...,BKG-201909713873_1,Everything the location was great the staff we...,"[(location, 0.8999999761581421), (Everything, ...",NaN


In [145]:
# data_s1_2['entity_score'] = data_s1_2.Sentence.map(str).apply(lambda x: get_entities_senti(x))



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [146]:
# data_s1_3['entity_score'] = data_s1_3.Sentence.map(str).apply(lambda x: get_entities_senti(x))



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [147]:
# data_s1_4['entity_score'] = data_s1_4.Sentence.map(str).apply(lambda x: get_entities_senti(x))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [160]:
data_11 = pd.concat([data_s1_1, data_s1_2, data_s1_3, data_s1_4])
data_11

,HB_SOCIAL_INSIGHT_REVIEW_ID,REVIEW_TXT,ID_SID,Sentence,entity_score
0,BKG-20190696110,Comfortable room. great location. good facilities,BKG-20190696110_1,Comfortable room,"[(room, 0.8999999761581421)]"
1,BKG-20190696110,Comfortable room. great location. good facilities,BKG-20190696110_2,great location,"[(location, 0.8999999761581421)]"
2,BKG-20190696110,Comfortable room. great location. good facilities,BKG-20190696110_3,good facilities,"[(facilities, 0.8999999761581421)]"
3,BKG-201909713799,Very good location great staff and great exper...,BKG-201909713799_1,Very good location great staff and great exper...,"[(location, 0.8999999761581421), (staff, 0.899..."
4,BKG-201909713799,Very good location great staff and great exper...,BKG-201909713799_2,Honestly nothings,"[(nothings, -0.8999999761581421)]"
...,...,...,...,...,...
1170,GGL-20190685608,Staff very friendly and helpful. Nice size roo...,GGL-20190685608_3,The breakfast was fine if you like a cooked b...,"[(breakfast, 0.699999988079071), (breakfast, 0..."
1171,GGL-20190685608,Staff very friendly and helpful. Nice size roo...,GGL-20190685608_4,Also the bananas were very over ripe,"[(bananas, -0.5)]"
1172,GGL-20190685608,Staff very friendly and helpful. Nice size roo...,GGL-20190685608_5,And most importantly no de caff tea bags,"[(caff tea bags, -0.4000000059604645)]"
1173,GGL-20190685608,Staff very friendly and helpful. Nice size roo...,GGL-20190685608_6,However overall a great stay for a decent price,"[(stay, 0.8999999761581421), (price, 0.8999999..."


In [162]:
data_11.set_index('ID_SID').entity_score.apply(pd.Series).stack().reset_index(name = 'entity_score')

,ID_SID,level_1,entity_score
0,BKG-20190696110_1,0,"(room, 0.8999999761581421)"
1,BKG-20190696110_2,0,"(location, 0.8999999761581421)"
2,BKG-20190696110_3,0,"(facilities, 0.8999999761581421)"
3,BKG-201909713799_1,0,"(location, 0.8999999761581421)"
4,BKG-201909713799_1,1,"(staff, 0.8999999761581421)"
...,...,...,...
2738,GGL-20190685608_3,4,"(offer, -0.10000000149011612)"
2739,GGL-20190685608_4,0,"(bananas, -0.5)"
2740,GGL-20190685608_5,0,"(caff tea bags, -0.4000000059604645)"
2741,GGL-20190685608_6,0,"(stay, 0.8999999761581421)"


In [0]:
data_12 = data_11.set_index('ID_SID').entity_score.apply(pd.Series).stack().reset_index(name = 'entity_score')

In [0]:
data_13 = data_12.merge(data_11[['ID_SID','Sentence']])

In [171]:
for index,i in enumerate(data_13.entity_score):
  data_13.loc[index, 'entity'] = i[0]
  data_13.loc[index, 'score'] = i[1]
    
    
data_13

,ID_SID,level_1,entity_score,Sentence,entity,score
0,BKG-20190696110_1,0,"(room, 0.8999999761581421)",Comfortable room,room,0.9
1,BKG-20190696110_2,0,"(location, 0.8999999761581421)",great location,location,0.9
2,BKG-20190696110_3,0,"(facilities, 0.8999999761581421)",good facilities,facilities,0.9
3,BKG-201909713799_1,0,"(location, 0.8999999761581421)",Very good location great staff and great exper...,location,0.9
4,BKG-201909713799_1,1,"(staff, 0.8999999761581421)",Very good location great staff and great exper...,staff,0.9
...,...,...,...,...,...,...
2738,GGL-20190685608_3,4,"(offer, -0.10000000149011612)",The breakfast was fine if you like a cooked b...,offer,-0.1
2739,GGL-20190685608_4,0,"(bananas, -0.5)",Also the bananas were very over ripe,bananas,-0.5
2740,GGL-20190685608_5,0,"(caff tea bags, -0.4000000059604645)",And most importantly no de caff tea bags,caff tea bags,-0.4
2741,GGL-20190685608_6,0,"(stay, 0.8999999761581421)",However overall a great stay for a decent price,stay,0.9


In [0]:
data_13 = data_13.drop(columns = 'level_1')

In [179]:
data_13 = data_13.reindex(columns = ['ID_SID', 'Sentence','entity_score', 'entity', 'score'])
data_13

,ID_SID,Sentence,entity_score,entity,score
0,BKG-20190696110_1,Comfortable room,"(room, 0.8999999761581421)",room,0.9
1,BKG-20190696110_2,great location,"(location, 0.8999999761581421)",location,0.9
2,BKG-20190696110_3,good facilities,"(facilities, 0.8999999761581421)",facilities,0.9
3,BKG-201909713799_1,Very good location great staff and great exper...,"(location, 0.8999999761581421)",location,0.9
4,BKG-201909713799_1,Very good location great staff and great exper...,"(staff, 0.8999999761581421)",staff,0.9
...,...,...,...,...,...
2738,GGL-20190685608_3,The breakfast was fine if you like a cooked b...,"(offer, -0.10000000149011612)",offer,-0.1
2739,GGL-20190685608_4,Also the bananas were very over ripe,"(bananas, -0.5)",bananas,-0.5
2740,GGL-20190685608_5,And most importantly no de caff tea bags,"(caff tea bags, -0.4000000059604645)",caff tea bags,-0.4
2741,GGL-20190685608_6,However overall a great stay for a decent price,"(stay, 0.8999999761581421)",stay,0.9


In [0]:
q = pd.ExcelWriter('OP_SentLevel.xlsx')
data_11.to_excel(q, sheet_name = 'Sent_level_entity_Score')
data_13.to_excel(q, sheet_name = 'ID_Sent_entity_Score')
q.save()